In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm 
import itertools
import sklearn.feature_selection
import sklearn.model_selection as ms
import pickle
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import datetime
import xgboost as xgb
from scipy.stats import skew
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, RidgeCV, LassoCV, ElasticNetCV
from scipy import stats
from sklearn.preprocessing import normalize
from scipy.stats import skew
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
plt.style.use('ggplot')
pd.options.display.max_columns = None
pd.options.display.max_rows = 600

In [5]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [6]:
# Adding 'type' columns that contains 'train' or 'test', and add empty SalePrice columns to the test part
train.insert(0, 'type', 'train')
test.insert(0,'type','test')
test['SalePrice'] = 0.0

# merging the two dataframes
merged = pd.concat([train,test],axis=0).reset_index(drop=True)
merged['SalePrice'] = merged['SalePrice'].astype('float')

# Data cleaning
import Dprocessv3
train = Dprocessv3.dataProcess(merged);

In [7]:
# numerical and categorical features + EXCLUDED FIRST
excl_features = merged[['Id','type','SalePrice']].columns
cat_features = merged.select_dtypes(include = ["object"]).columns.drop('type')
num_features = merged.select_dtypes(exclude = ["object"]).columns.drop(['Id','SalePrice'])

mergedNum = merged[num_features]
mergedCat = merged[cat_features]
mergedExc = merged[excl_features]

mergedNAtotal = merged.isnull().sum().sort_values(ascending=False)

print("NAs for numerical features in merged : " + str(mergedNum.isnull().values.sum()))

if mergedNum.isnull().values.sum() != 0:
    mergedNum = mergedNum.fillna(mergedNum.median())
    print("Remaining NAs for numerical features in merged : " + str(mergedNum.isnull().values.sum()))

NAs for numerical features in merged : 0


In [8]:
numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerics2 = []
for i in merged.columns:
    if merged[i].dtype in numeric_dtypes: 
        numerics2.append(i)

skew_features = merged[numerics2].apply(lambda x: skew(x)).sort_values(ascending=False)
skews = pd.DataFrame({'skew':skew_features})

In [9]:
high_skew = skew_features[skew_features > 0.5]
high_skew = high_skew
skew_index = high_skew.index

for i in skew_index:
    if i != 'SalePrice':
        merged[i]= boxcox1p(merged[i], boxcox_normmax(merged[i]+1))

        
skew_features2 = merged[numerics2].apply(lambda x: skew(x)).sort_values(ascending=False)
skews2 = pd.DataFrame({'skew':skew_features2})

/Users/macpro/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:3010: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den


In [10]:
# Create dummy features for categorical values via one-hot encoding

print("Remaining NAs for categorical features in merged : " + str(mergedCat.isnull().values.sum()))

print("NAs for categorical features in merged : " + str(mergedCat.isnull().values.sum()))

mergedCat = pd.get_dummies(mergedCat)


Remaining NAs for categorical features in merged : 0
NAs for categorical features in merged : 0


In [11]:
# rejoin excluded, categorical and numerical features
merged = pd.concat([mergedExc,mergedNum, mergedCat], axis = 1)

print("New number of features : " + str(merged.shape[1]))

New number of features : 335


In [12]:
merged.shape

(2916, 335)

In [13]:
train = merged[merged['type'] == 'train']
train.drop(columns=['Id','type'],inplace=True)
test = merged[merged['type'] == 'test']
test.drop(columns=['type','SalePrice'],inplace=True)

/Users/macpro/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [14]:
# taking the log of the saleprice

train['SalePrice'] = np.log(train['SalePrice'])
train['SalePrice'].describe()

/Users/macpro/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


count    1457.000000
mean       12.023714
std         0.399695
min        10.460242
25%        11.774520
50%        12.001505
75%        12.273731
max        13.534473
Name: SalePrice, dtype: float64

### Linear Regression

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
import sklearn.model_selection as ms
from scipy.stats import norm
from scipy.stats import skew
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.ensemble import (GradientBoostingRegressor , RandomForestRegressor,
                              BaggingRegressor,ExtraTreesRegressor,AdaBoostRegressor)
from scipy import stats
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error, make_scorer
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

pd.set_option('display.max_columns', 85)
pd.set_option('display.max_rows', 85)

In [ ]:
y = train.SalePrice

## Defining the train/test split (80/20)

In [15]:
house_features = train.drop('SalePrice',axis = 1)
prices = train['SalePrice']

x_train, x_test, y_train, y_test = ms.train_test_split(house_features, prices, 
                                                       test_size=0.2, random_state=111)

## Ridge

In [22]:
# 981 
# 0.12077
# R2: 0.9420
# trainRMSE: 0.096945
# testRMSE: 0.101837

ridcv = RidgeCV(alphas=np.linspace(1e-5,20,10000), normalize=False, scoring=None)

ridcv.fit(x_train,y_train)

trainRMSE = np.sqrt(mean_squared_error(y_true=y_train, y_pred=ridcv.predict(x_train)))
testRMSE = np.sqrt(mean_squared_error(y_true=y_test, y_pred=ridcv.predict(x_test)))

print('The best alpha is: {:.6f}\n\nThe ridge intercept is: {:.2f}\n\n{}\n\nThe model score is {:.4f}\n\nTrain RMSE: {:.6f}\n\nTest RMSE: {:.6f}\n\nDiff in RMSE: {:.3f}\n\n'.format(
    ridcv.alpha_,ridcv.intercept_, pd.Series(ridcv.coef_, index=x_train.columns), ridcv.score(x_train, y_train), trainRMSE,testRMSE,(testRMSE - trainRMSE)))

# pd.DataFrame({'Id': test['Id'], 'SalePrice': np.exp(ridcv.predict(test.drop(columns='Id')))}
#             ).to_csv('out {}.csv'.format(str(datetime.datetime.now())[:19].replace(':','.')
#                                            ), index=False)

The best alpha is: 14.095412

The ridge intercept is: 5.24

LotFrontage              0.000602
LotArea                  0.000002
OverallQual              0.050554
OverallCond              0.038554
YearBuilt                0.001866
YearRemodAdd             0.000751
MasVnrArea               0.000010
BsmtFinSF1               0.000058
BsmtFinSF2               0.000049
BsmtUnfSF               -0.000008
TotalBsmtSF              0.000099
1stFlrSF                 0.000090
2ndFlrSF                 0.000096
LowQualFinSF            -0.000022
GrLivArea                0.000164
BsmtFullBath             0.016849
BsmtHalfBath             0.003232
FullBath                 0.023480
HalfBath                 0.027196
BedroomAbvGr            -0.002121
KitchenAbvGr            -0.015578
TotRmsAbvGrd             0.001469
Fireplaces               0.002464
GarageYrBlt              0.000070
GarageCars               0.029411
GarageArea               0.000068
WoodDeckSF               0.000103
OpenPorchSF           

## Elastic-net

In [23]:
# 917
# 0.11994
# R2: 0.9359
# trainRMSE: 0.101935
# testRMSE: 0.100320
# diff: -0.002

enet = ElasticNetCV(l1_ratio=np.linspace(0.01,1,100), alphas=np.linspace(1e-5,20,1000), max_iter=1000, normalize=False, cv=10, n_jobs=-1)

enet.fit(x_train, y_train)

trainRMSE = np.sqrt(mean_squared_error(y_true=y_train, y_pred=enet.predict(x_train)))
testRMSE = np.sqrt(mean_squared_error(y_true=y_test, y_pred=enet.predict(x_test)))

print('The best alpha is: {:.6f}\n\nThe elastic-net intercept is: {:.2f}\n\n{}\n\nThe model score is {:.4f}\n\nTrain RMSE: {:.6f}\n\nTest RMSE: {:.6f}\n\nDiff in RMSE: {:.3f}\n\n'.format(
    enet.alpha_,enet.intercept_, pd.Series(enet.coef_, index=x_train.columns), enet.score(x_train, y_train), trainRMSE,testRMSE,(testRMSE - trainRMSE)))

# pd.DataFrame({'Id': test['Id'], 'SalePrice': np.exp(enet.predict(test.drop(columns='Id')))}
#             ).to_csv('out {}.csv'.format(str(datetime.datetime.now())[:19].replace(':','.')
#                                            ), index=False)

/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did no

/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did no

/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did no

/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did no

/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did no

...

/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did no

/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did no

/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did no

/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/macpro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did no

The best alpha is: 0.020030

The ridge intercept is: 4.78

LotFrontage              0.000626
LotArea                  0.000002
OverallQual              0.055121
OverallCond              0.042268
YearBuilt                0.001977
YearRemodAdd             0.000792
MasVnrArea               0.000016
BsmtFinSF1               0.000132
BsmtFinSF2               0.000116
BsmtUnfSF                0.000058
TotalBsmtSF              0.000039
1stFlrSF                 0.000220
2ndFlrSF                 0.000227
LowQualFinSF             0.000094
GrLivArea                0.000036
BsmtFullBath             0.014634
BsmtHalfBath             0.000000
FullBath                 0.017835
HalfBath                 0.021191
BedroomAbvGr            -0.002443
KitchenAbvGr            -0.009994
TotRmsAbvGrd             0.001033
Fireplaces               0.006512
GarageYrBlt              0.000144
GarageCars               0.026263
GarageArea               0.000079
WoodDeckSF               0.000101
OpenPorchSF            

## Lasso

In [25]:
#  alpha           UJ range              np.linspace(1e-4,1e-2,1000)
#  CV          100         10          100
# R2:        0.9366      0.9399      0.9374
# trainRMSE: 0.101363    0.098698    0.100718
# testRMSE:  0.100012    0.100196    0.099977
# diff:b       0.00       0.00         0.00

lassocv = LassoCV(alphas=np.linspace(1e-4,1e-3,1000), normalize=False,
                      max_iter=10000, n_jobs=-1, cv=100)

lassocv.fit(x_train, y_train)

trainRMSE = np.sqrt(mean_squared_error(y_true=y_train, y_pred=lassocv.predict(x_train)))
testRMSE = np.sqrt(mean_squared_error(y_true=y_test, y_pred=lassocv.predict(x_test)))

print('The best alpha is: {:.6f}\n\nThe lasso intercept is: {:.2f}\n\n{}\n\nThe model score is {:.4f}\n\nTrain RMSE: {:.6f}\n\nTest RMSE: {:.6f}\n\nDiff in RMSE: {:.3f}\n\n'.format(
    lassocv.alpha_,lassocv.intercept_, pd.Series(lassocv.coef_, index=x_train.columns), lassocv.score(x_train, y_train), trainRMSE,testRMSE,(testRMSE - trainRMSE)))

# pd.DataFrame({'Id':test['Id'],'SalePrice': np.exp(lassocv.predict(test.drop(columns='Id')))}
#             ).to_csv('out {}.csv'.format(str(datetime.datetime.now())[:19].replace(':','.')
#                                            ), index=False)

The best alpha is: 0.000646

The ridge intercept is: 4.83

LotFrontage              6.375575e-04
LotArea                  1.861251e-06
OverallQual              5.469325e-02
OverallCond              4.064895e-02
YearBuilt                1.860746e-03
YearRemodAdd             8.221041e-04
MasVnrArea               1.469642e-05
BsmtFinSF1               1.338251e-04
BsmtFinSF2               1.028998e-04
BsmtUnfSF                5.621080e-05
TotalBsmtSF              3.875790e-05
1stFlrSF                 2.213459e-04
2ndFlrSF                 2.243127e-04
LowQualFinSF             1.163900e-04
GrLivArea                3.587795e-05
BsmtFullBath             1.261940e-02
BsmtHalfBath             0.000000e+00
FullBath                 1.347639e-02
HalfBath                 1.663858e-02
BedroomAbvGr            -2.848209e-03
KitchenAbvGr            -7.554874e-03
TotRmsAbvGrd             0.000000e+00
Fireplaces               2.816351e-03
GarageYrBlt              2.360013e-04
GarageCars               3.01

## Random Forest

In [31]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=1000,max_depth=7,max_features='auto', min_samples_split=2, min_samples_leaf=1,n_jobs=-1)
rf.fit(x_train, y_train)

scores = ms.cross_val_score(rf, x_train, y_train, cv=10)

print(scores.mean())

trainRMSE = np.sqrt(mean_squared_error(y_true=y_train, y_pred=rf.predict(x_train)))
testRMSE = np.sqrt(mean_squared_error(y_true=y_test, y_pred=rf.predict(x_test)))

print('\nThe model score is {:.4f}\n\nTrain RMSE: {:.6f}\n\nTest RMSE: {:.6f}\n\nDiff in RMSE: {:.3f}\n\n'.format(
    rf.score(x_train, y_train), trainRMSE,testRMSE,(testRMSE - trainRMSE)))


# 1000 trees
# 0.8395225412896815
# R2: 0.9517
# Train RMSE: 0.088419
# Test RMSE: 0.129310
# Diff in RMSE: 0.041

0.8641562863923543

The model score is 0.9517

Train RMSE: 0.088419

Test RMSE: 0.129310

Diff in RMSE: 0.041


